 Myself [Harsh Patel](https://www.linkedin.com/in/harshp1035/) creating an ML based Recommendation Engine in collaboration with [Mr. Rocky Jagtiani](https://www.linkedin.com/today/author/rocky-jagtiani-3b390649/)
 
> This is a simple Data Science project on Movies Recommendation System which recommends you the movie based on the Review of previous movie.

> Dataset: tmdb_5000_credits.csv,tmdb_5000_movies.csv from kaggle itself

> Tech Stack used: pandas, Scikit-learn,Python

> Recommended links : 

> https://datascience.suvenconsultants.com  ( For DS / AI / ML )

> https://monster.suvenconsultants.com  ( For Web development )

Recommender systems are among the most popular applications of data science today. They are used to predict the "rating" or "preference" that a user would give to an item. Almost every major tech company has applied them in some form. Amazon uses it to suggest products to customers, YouTube uses it to decide which video to play next on autoplay, and Facebook uses it to recommend pages to like and people to follow.

Recommender systems have also been developed to explore research articles and experts, collaborators, and financial services. 

Recommender systems can be classified into Two types:

> **Content-based recommenders**: suggest similar items based on a particular item. This system uses item metadata, such as genre, director, description, actors, etc. for movies, to make these recommendations. The general idea behind these recommender systems is that if a person likes a particular item, he or she will also like an item that is similar to it. And to recommend that, it will make use of the user's past item metadata. A good example could be YouTube, where based on your history, it suggests you new videos that you could potentially watch.

> **Collaborative filtering engines**: these systems are widely used, and they try to predict the rating or preference that a user would give an item-based on past ratings and preferences of other users. Collaborative filters do not require item metadata like its content-based counterparts.

Here we are going to implement **Content Based Filtering**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import Pandas
import pandas as pd

# Loading Data sets
full_url='/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv'

full_url1='/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv'

credits = pd.read_csv(full_url)
movies=pd.read_csv(full_url1)

In [ ]:
# Printing 1st 5 elements of credits dataset
credits.head()

In [ ]:
# Printing 1st 5 elements of movies dataset
movies.head(2)

In [ ]:
# Printing the shapes of both the datasets
print("Credits:",credits.shape)
print("Movies:",movies.shape)

In [ ]:
# Renaming the column of credits data set
credits_renamed=credits.rename(index=str,columns={'movie_id':'id'})
credits_renamed.head()

In [ ]:
# Merging both data sets
merge=movies.merge(credits_renamed,on='id')
merge.head()
#to understand merge ,  merge work as joins

In [ ]:
# Dropping unnecessary columns 
cleaned=merge.drop(columns=['homepage','title_x','title_y','status','production_countries'])
cleaned.head()
## one aspect of cleaning is dropping unnecessary columns which won't help the task at hand

In [ ]:
cleaned['overview'].head()

In [ ]:
cleaned['overview'].isnull().sum() 
#no nan or null values for overview

In [ ]:
#Replace NaN with an empty string
cleaned['overview'] = cleaned['overview'].fillna('')

In [ ]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english',ngram_range=(1,3),min_df=3,analyzer='word')
#refer : http://www.tfidf.com/


#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(cleaned['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

In [ ]:
#linear_kernal is little bit more faster tha cosine_similarity but this dataset is not so big. So we can use cosine_similarity
#from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
#cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

from sklearn.metrics.pairwise import cosine_similarity
 
# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [ ]:
print(cosine_sim.shape)
print(cosine_sim[0])
print(cosine_sim[1])

We are going to define a function that takes in a movie title as an input and outputs a list of 10 most similar movies. Firstly, for this we need a reverse mapping fo movies titles and DataFrame incides. In outher words, we need a mechanism to identify the index of movie in out metadata DataFrame, given its title.

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(cleaned.index, index=cleaned['original_title']).drop_duplicates()


In [ ]:
indices[0:5]

In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    print(sim_scores[0:5])
    
    print("--------------------------")

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]
    print(sim_scores[0:5])

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return cleaned['original_title'].iloc[movie_indices]

In [ ]:
# Getting the recommendation
get_recommendations('Avatar')

In [ ]:
get_recommendations('The Dark Knight Rises')

# Enhancements Possible

In [ ]:
cleaned.columns

In [ ]:
## From your new features, cast, crew, and keywords, 
## you need to extract the three most important actors, 
## the director and the keywords associated with that movie.

## But first things first, your data is present in the form of "stringified" lists. 
## You need to convert them into a way that is usable for you.

# Parse the stringified features into their corresponding python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']

for feature in features:
    cleaned[feature] = cleaned[feature].apply(literal_eval)
    
## about literal_eval()    
## https://stackoverflow.com/questions/15197673/using-pythons-eval-vs-ast-literal-eval

In [ ]:
## lets see the data stored for the 0th movie.  
cleaned['crew'].values[0]

## Notice : its an list of dict objects.


In [ ]:
## user defn function , which finds the name of the director from 
## list od crew members

def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan


In [ ]:
def get_list(x):
    
    if isinstance(x, list):
        names = [i['name'] for i in x]
    
    #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[ : 3 ]
        return names

    #Return empty list in case of missing/malformed data
    return []


In [ ]:
# Define new director, cast, genres and keywords features that are in a suitable form.
cleaned['director'] = cleaned['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']

for feature in features:
    cleaned[feature] = cleaned[feature].apply(get_list)


In [ ]:
# Print the new features of the first 3 films
cleaned[['original_title', 'cast', 'director', 'keywords', 'genres']].head(3)

In [ ]:
#Akshay Kumar will be converted to akshaykumar and Akshay khanna -> akshaykhanna.
#so that the vectorizer will consider akshaykumar and akhsaykhanna as different people.
##replacing the blank space to no space askhay Kumar as akshaykumar to reduce the duplication

# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''


In [ ]:
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    cleaned[feature] = cleaned[feature].apply(clean_data)


In [ ]:
#it will create a metadata(in string), combination of keyword,cast, director and genres. to feed it to vectorizer.
def create_metadata(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])


In [ ]:
# Create a new metadata feature
cleaned['metadata'] = cleaned.apply(create_metadata, axis=1)

In [ ]:
cleaned[['metadata']].head(2)

In [ ]:
## Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')

count_matrix = count.fit_transform(cleaned['metadata'])


In [ ]:
count_matrix.shape

In [ ]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of your main DataFrame and construct reverse mapping as before

## cleaned = cleaned.reset_index()
indices = pd.Series(cleaned.index, index = cleaned['original_title'])
indices[:2]

In [ ]:
get_recommendations("The Dark Knight Rises", cosine_sim2)

I would like to humbly and sincerely thank my mentor [Rocky Jagtiani](https://www.linkedin.com/today/author/rocky-jagtiani-3b390649/). He is more of a friend to me then mentor. The Machine Learning course taught by him and various projects we did and are still doing is the best way to learn and skill in Data Science field. See https://datascience.suvenconsultants.com once for more.